# Generating the data
1. Generate the Heterogeneous graph
2. Generate the feature set from the clinical notes.
3. Generate the Labels
4. Generate the k-metapath-based similarity matrices
5. Convert the As to edge-based.

In [1]:
import os
import torch
import networkx as nx

def save_list_as_pickle(L, given_path, file_name):
    import pickle
    print(f'saving to {given_path}/{file_name}.pkl')
    with open(f'{given_path}/{file_name}.pkl', 'wb') as file:
        pickle.dump(L, file)


num_Diseases    = int(os.getenv('NUM_DISEASES', 203))  
DISEASE_FILE    = os.getenv('DISEASE_FILE', f'DMPLB')  
similarity_type = os.getenv('similarity_type', 'PC')  # options are PC: PathCount, SPS: Symmetric PathSim

num_Sample      = int(os.getenv('num_Sample', 500))  
r_u_sampling    = os.getenv('r_u_sampling', 'True')  

if r_u_sampling=='True':
    sampling = True
else:
    sampling = False

disease_data_path = '/lustre/home/almusawiaf/PhD_Projects/HGNN_Project2/Data'

print(num_Diseases, DISEASE_FILE, similarity_type, num_Sample, sampling)


203 DMPLB PC 500 True


In [2]:
from module1 import generating_HG as G_class
from module1 import XY_preparation as XY
from module1 import meta_path as MP
# =================================================================================
saving_path = f'{disease_data_path}/{num_Diseases}_Diagnoses/{DISEASE_FILE}/{num_Sample}'

for p in ['HGNN_data', 'clinical_items', 'GMLs', 'OHV', 'PSGs', 'SNFs']:
    os.makedirs(f'{saving_path}/{p}', exist_ok=True)

saving_path = f'{disease_data_path}/{num_Diseases}_Diagnoses/{DISEASE_FILE}/{num_Sample}/HGNN_data'
# =================================================================================
HG_instance = G_class.Generate_HG(sampling = sampling, num_Patients = num_Sample)
HG = HG_instance.HG

G_class.G_statistics(HG)
# =================================================================================
if not sampling:
    num_Sample = len(HG_instance.Patients)
# ============================ Extracting Patient-based X and Y =================================
XY_inst = XY.XY_preparation(HG)
X = XY_inst.X
Y = XY_inst.Y
# ============================ Extracting Visit-based X and Y =================================
XY_inst = XY.XY_preparation(HG)
XV = XY_inst.X_visit
YV = XY_inst.Y_visit
# ======================= Computing the Meta Path based Similarities ======================
MP_inst = MP.Meta_path(HG, similarity_type = 'PC', saving_path = saving_path)
# ==================================== SAVING =============================================
nx.write_gml(HG, f'{saving_path}/HG.gml')
save_list_as_pickle(MP_inst.Nodes,   saving_path, 'Nodes')
# ==================================== Saving X and Y  (patient-based) ============================
torch.save(X, f'{saving_path}/X.pt')
torch.save(Y, f'{saving_path}/Y.pt')
# ==================================== Saving X and Y (visit-based) =================================
torch.save(X, f'{saving_path}/XV.pt')
torch.save(Y, f'{saving_path}/YV.pt')


Loading the dataframes...

We are SAMPLING

Use the patients inside the new DataFrame....
Dropping NaN visits
General Information:
---------------------------
Number of Patients = 500
Number of Diagnosis = 203
Number of procedures = 71
Number of Medication = 124
Number of Lab tests  = 461
Number of MicroBio   = 49
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Splitting lab tests


/lustre/home/almusawiaf/PhD_Projects/HGNN_Project2/Data_Generation/module1/generating_HG.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lab_df.loc[:, 'ITEMID_FLAG'] = lab_df['ITEMID'].astype(str) + '_' + lab_df['FLAG'].astype(str)


        HADM_ID     ITEMID_FLAG
94533  156461.0  50971_abnormal
94534  156461.0    50983_normal
94535  156461.0    50868_normal
94536  156461.0  50882_abnormal
94537  156461.0  50883_abnormal
94538  156461.0    50884_normal
94539  156461.0    50885_normal
94540  156461.0    50902_normal
94541  156461.0  50971_abnormal
94542  156461.0    50983_normal
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Extracting bipartite networks...

Extracting and adding data of Visits
SUBJECT_ID HADM_ID C V
      ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
1582    2140         175   159223      1.0       806
1583    2141         175   159223      2.0       805
1584    2142         175   159223      3.0       413
1585    2143         175   159223      4.0       427
1586    2144         175   159223      5.0       V42

Extracting and adding data of Diagnosis
HADM_ID ICD9_CODE V D
      ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
1582    2140         175   159223      1.0       806
1583   